In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import time

#Visualization
import matplotlib.pyplot as plt 
import altair as alt
import seaborn as sns

#Data preparation
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#Models
from sklearn.linear_model import LinearRegression                             
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
import xgboost as xgb

#Metrics
from sklearn.metrics import (explained_variance_score, max_error, 
                             mean_absolute_error, mean_squared_error, 
                             median_absolute_error, r2_score)

from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.pipeline import Pipeline
from collections import Counter

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

### Import data

In [2]:
# Base de Treino
df_train = pd.read_csv("train.csv")

# Base de Teste
df_test = pd.read_csv("test.csv")
df_test_n = pd.read_csv("test.csv")

In [3]:
df_train.shape, df_train.NU_INSCRICAO.nunique(), df_train.NU_ANO.value_counts()

((13730, 167), 13730, 2016    13730
 Name: NU_ANO, dtype: int64)

In [4]:
# Temos apenas 47 features na base de teste, portanto é interessante treinar apenas com features em comum

x_features = set.intersection(set(df_test.columns.to_list()), set(df_train.columns.to_list()))

target = df_train.IN_TREINEIRO
df_train = df_train[x_features] # 13730, 43
df_train['istrain'] = 1
df_test = df_test[x_features] # 4576, 43

df_full = pd.concat([df_train,df_test], axis = 0, sort = False).reset_index(drop = True)
df_full.istrain.fillna(0,inplace=True)

df_num = df_full.select_dtypes(exclude='object') # 18306, 33
df_str = df_full.select_dtypes(include='object') # 18306, 15

In [5]:
print('Int64:')
print(df_full.select_dtypes(include='Int64').columns.values)
print('\nFloat64:')
print(df_full.select_dtypes(include='Float64').columns.values)
print('\nStrings:')
print(df_full.select_dtypes(include='object').columns.values)

Int64:
['CO_UF_RESIDENCIA' 'TP_PRESENCA_CH' 'TP_PRESENCA_LC' 'IN_BAIXA_VISAO'
 'TP_NACIONALIDADE' 'TP_COR_RACA' 'NU_IDADE' 'IN_IDOSO' 'IN_DISCALCULIA'
 'TP_ESCOLA' 'IN_CEGUEIRA' 'IN_SABATISTA' 'IN_DISLEXIA' 'IN_GESTANTE'
 'TP_ANO_CONCLUIU' 'TP_PRESENCA_MT' 'IN_SURDEZ' 'TP_ST_CONCLUSAO'
 'TP_LINGUA' 'TP_PRESENCA_CN']

Float64:
['TP_DEPENDENCIA_ADM_ESC' 'NU_NOTA_CN' 'NU_NOTA_COMP3' 'TP_STATUS_REDACAO'
 'NU_NOTA_CH' 'NU_NOTA_COMP5' 'NU_NOTA_REDACAO' 'NU_NOTA_COMP4'
 'NU_NOTA_COMP2' 'NU_NOTA_LC' 'TP_ENSINO' 'NU_NOTA_COMP1' 'istrain']

Strings:
['Q047' 'Q027' 'Q025' 'Q001' 'Q006' 'Q002' 'TP_SEXO' 'Q026' 'Q024'
 'NU_INSCRICAO' 'SG_UF_RESIDENCIA']


### Data Cleaning

In [6]:
#Returns null values (%)
def get_nans(df):
    nan_dic = {}
    for col in df.columns:
        if df[col].isnull().any() == True:
            nan_dic[col] = df[col].isnull().sum()
    return pd.DataFrame({
        'Feature': list(nan_dic.keys()),
        'Nulls': list(nan_dic.values()),
        'Percent': np.round((np.array(list(nan_dic.values())) / df.shape[0])*100, decimals = 1)
    }).sort_values('Nulls',ascending = False)

#Returns 0 values (%)
def get_zeros(df):
    zero_dic = {}
    for col in df.columns:
        if (df[col] == 0).sum() > 0:
            zero_dic[col] = (df[col] == 0).sum()
    return pd.DataFrame({'Feature': list(zero_dic.keys()),
                        'Zeros': list(zero_dic.values()),
                        'Percent': np.round((np.array(list(zero_dic.values())) / df.shape[0])*100, decimals = 1)
    }).sort_values('Zeros',ascending = False)

In [7]:
#Situação das variáveis numéricas

print('Sumário dos nulos:')
print(get_nans(df_num))
print('')
print('Sumário dos zeros:')
print(get_zeros(df_num))

Sumário dos nulos:
                   Feature  Nulls  Percent
0   TP_DEPENDENCIA_ADM_ESC  12592     68.8
10               TP_ENSINO  12592     68.8
2            NU_NOTA_COMP3   4767     26.0
3        TP_STATUS_REDACAO   4767     26.0
5            NU_NOTA_COMP5   4767     26.0
6          NU_NOTA_REDACAO   4767     26.0
7            NU_NOTA_COMP4   4767     26.0
8            NU_NOTA_COMP2   4767     26.0
9               NU_NOTA_LC   4767     26.0
11           NU_NOTA_COMP1   4767     26.0
1               NU_NOTA_CN   4501     24.6
4               NU_NOTA_CH   4501     24.6

Sumário dos zeros:
             Feature  Zeros  Percent
12    IN_DISCALCULIA  18299    100.0
14       IN_CEGUEIRA  18299    100.0
16       IN_DISLEXIA  18298    100.0
11          IN_IDOSO  18297    100.0
20         IN_SURDEZ  18295    100.0
17       IN_GESTANTE  18283     99.9
5     IN_BAIXA_VISAO  18273     99.9
15      IN_SABATISTA  18126     99.0
18   TP_ANO_CONCLUIU   9578     52.3
22         TP_LINGUA   7365     

In [8]:
#Situação das variáveis categóricas

print('Sumário dos nulos:')
print(get_nans(df_str))

Sumário dos nulos:
  Feature  Nulls  Percent
0    Q027   9810     53.6


In [9]:
df_full.isna().sum()

CO_UF_RESIDENCIA              0
Q047                          0
TP_DEPENDENCIA_ADM_ESC    12592
NU_NOTA_CN                 4501
Q027                       9810
NU_NOTA_COMP3              4767
TP_PRESENCA_CH                0
TP_STATUS_REDACAO          4767
TP_PRESENCA_LC                0
NU_NOTA_CH                 4501
IN_BAIXA_VISAO                0
TP_NACIONALIDADE              0
Q025                          0
Q001                          0
TP_COR_RACA                   0
NU_IDADE                      0
Q006                          0
NU_NOTA_COMP5              4767
NU_NOTA_REDACAO            4767
NU_NOTA_COMP4              4767
IN_IDOSO                      0
IN_DISCALCULIA                0
NU_NOTA_COMP2              4767
TP_ESCOLA                     0
IN_CEGUEIRA                   0
IN_SABATISTA                  0
IN_DISLEXIA                   0
IN_GESTANTE                   0
TP_ANO_CONCLUIU               0
Q002                          0
TP_PRESENCA_MT                0
TP_SEXO 

In [10]:
#Cleaner

class Cleaner:
    
    from sklearn.preprocessing import MinMaxScaler
    
    def __init__(self, dataframe, bin_list = [], scaler_list = [], ordinal_list = [], dummy_list = [], clean_nan = False):    
        self.dataframe = dataframe
        self.bin_list = bin_list
        self.scaler_list = scaler_list
        self.ordinal_list = ordinal_list
        self.dummy_list = dummy_list
        self.clean_nan = clean_nan
    
    def transform(self):    
        dataframe = self.dataframe
        dataframe_cols = dataframe.columns.values.tolist()

        #Binariza
        if bool(self.bin_list):
            for col in self.bin_list:
                dataframe[col] = [0 if x == 0 else 1 for x in dataframe[col]]

        #Escalona(MinMax)
        if bool(self.scaler_list):
            scaler = MinMaxScaler()
            dataframe.loc[:,self.scaler_list] = scaler.fit_transform(dataframe.loc[:,self.scaler_list])

        #Label Ordinal  
        if bool(self.ordinal_list):
            for col in self.ordinal_list:
                dataframe[col] = dataframe[col].map({'M':13, 'K':12, 'L':11, 'J':10, 'I':9, 'H':8, 'G':7,
                                                     'F':6, 'E':5, 'D':4, 'C':3, 'B':2, 'A':1, np.nan:0})


        if bool(self.dummy_list):
            dataframe = pd.get_dummies(dataframe, columns=self.dummy_list, drop_first=True)

        if self.clean_nan:
            for col in dataframe.select_dtypes(exclude='object').columns.tolist():
                dataframe[col].fillna(dataframe[col].median(), inplace = True)

      
        return dataframe

In [11]:
df_full.head()

,CO_UF_RESIDENCIA,Q047,TP_DEPENDENCIA_ADM_ESC,NU_NOTA_CN,Q027,NU_NOTA_COMP3,TP_PRESENCA_CH,TP_STATUS_REDACAO,TP_PRESENCA_LC,NU_NOTA_CH,...,TP_ST_CONCLUSAO,NU_NOTA_LC,TP_ENSINO,Q024,NU_INSCRICAO,SG_UF_RESIDENCIA,TP_LINGUA,TP_PRESENCA_CN,NU_NOTA_COMP1,istrain
0,43,A,NaN,436.3,H,120.0,1,1.0,1,495.4,...,1,581.2,NaN,A,ed50e8aaa58e7a806c337585efee9ca41f1eb1ad,RS,1,1,120.0,1.0
1,23,A,2.0,474.5,NaN,120.0,1,1.0,1,544.1,...,2,599.0,1.0,A,2c3acac4b33ec2b195d77e7c04a2d75727fad723,CE,1,1,140.0,1.0
2,23,A,NaN,NaN,NaN,NaN,0,NaN,0,NaN,...,3,NaN,NaN,A,f4545f8ccb9ff5c8aad7d32951b3f251a26e6568,CE,1,0,NaN,1.0
3,33,D,NaN,NaN,F,NaN,0,NaN,0,NaN,...,1,NaN,NaN,C,3d6ec248fef899c414e77f82d5c6d2bffbeaf7fe,RJ,0,0,NaN,1.0
4,13,A,NaN,NaN,F,NaN,0,NaN,0,NaN,...,1,NaN,NaN,A,bf896ac8d3ecadd6dba1dfbf50110afcbf5d3268,AM,1,0,NaN,1.0


In [12]:

#Variáveis que são ordinais
ordinal_list = ['Q001','Q006','Q047', 'Q025', 'Q026', 'Q002', 'Q024', 'Q027']

dummy_list = ['SG_UF_RESIDENCIA', 'TP_SEXO', ]

#Instânciando cleaner
cleaner = Cleaner(df_full,
                  ordinal_list = ordinal_list,
                  dummy_list = dummy_list,
                  clean_nan = True)

print(f'Shape antes do dummify: {df_full.shape}')

#Aplicando transformações
df_full = cleaner.transform()

#Shape
print(f'Shape após o dummify: {df_full.shape}')

Shape antes do dummify: (18300, 44)
Shape após o dummify: (18300, 69)


### Exploratory Data Analysis

In [13]:
target.value_counts()

0    11947
1     1783
Name: IN_TREINEIRO, dtype: int64

In [14]:
df_full.drop('NU_INSCRICAO', axis = 1, inplace = True)

### Scaling

In [15]:
def scaling(df):
    scaler = StandardScaler()
    for col in df.columns:

        if col == 'istrain':
            pass

        elif df[col].unique().tolist() != [0,1]:
            df[col] = scaler.fit_transform(df[[col]])
    return df

#Aplicando scaling
df_full = scaling(df_full)

In [16]:
df_full.head()

,CO_UF_RESIDENCIA,Q047,TP_DEPENDENCIA_ADM_ESC,NU_NOTA_CN,Q027,NU_NOTA_COMP3,TP_PRESENCA_CH,TP_STATUS_REDACAO,TP_PRESENCA_LC,NU_NOTA_CH,...,SG_UF_RESIDENCIA_RJ,SG_UF_RESIDENCIA_RN,SG_UF_RESIDENCIA_RO,SG_UF_RESIDENCIA_RR,SG_UF_RESIDENCIA_RS,SG_UF_RESIDENCIA_SC,SG_UF_RESIDENCIA_SE,SG_UF_RESIDENCIA_SP,SG_UF_RESIDENCIA_TO,TP_SEXO_M
0,1.234775,-0.388067,-0.201491,-0.547712,1.932472,0.329729,0.568101,-0.122407,0.590147,-0.540310,...,0,0,0,0,4.857651,0,0,0,0,1.263519
1,-0.791070,-0.388067,-0.201491,0.068879,-0.755469,0.329729,0.568101,-0.122407,0.590147,0.221495,...,0,0,0,0,-0.205861,0,0,0,0,-0.791440
2,-0.791070,-0.388067,-0.201491,-0.166781,-0.755469,0.329729,-1.750938,-0.122407,-1.684908,0.025960,...,0,0,0,0,-0.205861,0,0,0,0,-0.791440
3,0.221852,2.529208,-0.201491,-0.166781,1.260486,0.329729,-1.750938,-0.122407,-1.684908,0.025960,...,1,0,0,0,-0.205861,0,0,0,0,-0.791440
4,-1.803993,-0.388067,-0.201491,-0.166781,1.260486,0.329729,-1.750938,-0.122407,-1.684908,0.025960,...,0,0,0,0,-0.205861,0,0,0,0,1.263519


In [17]:
df_full.shape

(18300, 68)

### Baseline model

In [19]:
#Separando
df_train = df_full.loc[df_full.istrain == 1]
df_test = df_full.loc[df_full.istrain == 0]

print(f'Train shape: {df_train.shape}')
print(f'Test shape: {df_test.shape}')
print(f'Target shape: {target.shape}')

Train shape: (13730, 68)
Test shape: (4570, 68)
Target shape: (13730,)


In [20]:
y = target.fillna(0)
X = df_train

In [21]:
# Split into training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=42)


In [26]:
from sklearn.ensemble import RandomForestClassifier

reg = RandomForestClassifier(max_depth = 7, random_state= 42).fit(X_train, y_train)
reg.score(X_train, y_train), reg.score(X_test, y_test)

(0.9946935802726043, 0.9941733430444283)

**Um modelo baseline é importante para identificarmos o ponto de partida do modelo. Devemos aprimorar essa abordagem e comparar os futuros scores com este, realizado com pouco esforço de feature selection, feature engineering e tuning de hiperparâmetros.**

In [27]:
df_full.head()

,CO_UF_RESIDENCIA,Q047,TP_DEPENDENCIA_ADM_ESC,NU_NOTA_CN,Q027,NU_NOTA_COMP3,TP_PRESENCA_CH,TP_STATUS_REDACAO,TP_PRESENCA_LC,NU_NOTA_CH,...,SG_UF_RESIDENCIA_RJ,SG_UF_RESIDENCIA_RN,SG_UF_RESIDENCIA_RO,SG_UF_RESIDENCIA_RR,SG_UF_RESIDENCIA_RS,SG_UF_RESIDENCIA_SC,SG_UF_RESIDENCIA_SE,SG_UF_RESIDENCIA_SP,SG_UF_RESIDENCIA_TO,TP_SEXO_M
0,1.234775,-0.388067,-0.201491,-0.547712,1.932472,0.329729,0.568101,-0.122407,0.590147,-0.540310,...,0,0,0,0,4.857651,0,0,0,0,1.263519
1,-0.791070,-0.388067,-0.201491,0.068879,-0.755469,0.329729,0.568101,-0.122407,0.590147,0.221495,...,0,0,0,0,-0.205861,0,0,0,0,-0.791440
2,-0.791070,-0.388067,-0.201491,-0.166781,-0.755469,0.329729,-1.750938,-0.122407,-1.684908,0.025960,...,0,0,0,0,-0.205861,0,0,0,0,-0.791440
3,0.221852,2.529208,-0.201491,-0.166781,1.260486,0.329729,-1.750938,-0.122407,-1.684908,0.025960,...,1,0,0,0,-0.205861,0,0,0,0,-0.791440
4,-1.803993,-0.388067,-0.201491,-0.166781,1.260486,0.329729,-1.750938,-0.122407,-1.684908,0.025960,...,0,0,0,0,-0.205861,0,0,0,0,1.263519


**1 Submission: XGBOOST**

In [28]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [29]:
xgb.score(X_train, y_train), xgb.score(X_test, y_test)

(0.9998959525543648, 0.9961155620296188)

In [33]:
# Submit 1
y_sub1 = xgb.predict(df_test) # Score: 99.69

pd.DataFrame({'NU_INSCRICAO':df_test_n.NU_INSCRICAO,'IN_TREINEIRO':y_sub1}).to_csv("answer.csv", index= False)

**2 Submission: Feature Selection (PCA) + XGBOOST**

In [34]:
from sklearn.decomposition import PCA

In [63]:
pca = PCA(n_components=40)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [64]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train_pca, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [65]:
X_train_pca.shape, X_test_pca.shape

((9611, 40), (4119, 40))

In [66]:
xgb.score(X_train_pca, y_train), xgb.score(X_test_pca, y_test)

(0.9998959525543648, 0.9803350327749454)

In [68]:
# Submit 2
df_test_pca = pca.transform(df_test)
y_sub2 = xgb.predict(df_test_pca) # Score: 98.16

pd.DataFrame({'NU_INSCRICAO':df_test_n.NU_INSCRICAO,'IN_TREINEIRO':y_sub2}).to_csv("answer.csv", index= False)

**3 Submission: Feature Selection (importance) + XGBOOST**

In [69]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [72]:
xgb_importances = pd.DataFrame({'columns':X_train.columns, 'importances':xgb.feature_importances_})

In [93]:
features = xgb_importances.sort_values(by='importances', ascending = False)['columns'].head(5).values

In [94]:
xgb = XGBClassifier()
xgb.fit(X_train[features], y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [95]:
xgb.score(X_train[features], y_train), xgb.score(X_test[features], y_test)

(0.9971907189678494, 0.9956300072833212)

In [98]:
# Submit 3
y_sub3 = xgb.predict(df_test[features]) # Score: 99.71

pd.DataFrame({'NU_INSCRICAO':df_test_n.NU_INSCRICAO,'IN_TREINEIRO':y_sub3}).to_csv("answer3.csv", index= False)

**4 Submission: AutoML**

In [99]:
import h2o
from h2o.automl import H2OAutoML

h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.251-b08, mixed mode)
  Starting server from C:\ProgramData\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\laris\AppData\Local\Temp\tmp3a1wwmul
  JVM stdout: C:\Users\laris\AppData\Local\Temp\tmp3a1wwmul\h2o_laris_started_from_python.out
  JVM stderr: C:\Users\laris\AppData\Local\Temp\tmp3a1wwmul\h2o_laris_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.7
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_laris_d40d9i
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.514 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [ ]:
x = X_train.columns.to_list()
y = 'IN_TREINEIRO'
df = pd.concat([X_train, y_train], axis = 1)
hf = h2o.H2OFrame(df)

# Run AutoML for 10 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=10, seed=1)
aml.train(x= x, y= y, training_frame= hf)

# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |
00:34:21.936: AutoML: XGBoost is not available; skipping it.

████████████████████

In [158]:
# The leader model is stored here: aml.leader

hf_X_test = h2o.H2OFrame(X_test)

y_pred = aml.predict(hf_X_test)
y_pred = y_pred.as_data_frame().predict
r2_score(y_pred.apply(f), y_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [186]:
# Submit 5

hf_df_test = h2o.H2OFrame(df_test)

y_sub5 = aml.predict(hf_df_test) # Score: 9.83
y_sub5 = y_sub5.as_data_frame().predict

pd.DataFrame({'NU_INSCRICAO':df_test_n.NU_INSCRICAO,'NU_NOTA_MT':pd.Series(y_sub5).apply(f)}).to_csv("answer.csv", index= False)

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%
